In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [ ]:
!du -hcs /kaggle/input/leash-BELKA/train.parquet

In [ ]:
!ls

In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 36.6 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 39.5 MB/s eta 0:00:0000:0100:01m


In [4]:
import duckdb
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

In [5]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mol1 = con.query(f"""(SELECT DISTINCT(buildingblock1_smiles) as buildingblock1_smiles
                        FROM parquet_scan('{train_path}')
                        )""").df()

mol2 = con.query(f"""(SELECT DISTINCT(buildingblock2_smiles) as buildingblock1_smiles
                        FROM parquet_scan('{train_path}')
                        )""").df()

mol3 = con.query(f"""(SELECT DISTINCT(buildingblock3_smiles) as buildingblock1_smiles
                        FROM parquet_scan('{train_path}')
                        )""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
mol1.shape

(271, 1)

In [7]:
mol2.shape

(693, 1)

In [8]:
mol3.shape

(872, 1)

In [9]:
mol1.head()

,buildingblock1_smiles
0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21
1,CC(C)(C)OC(=O)CCC(NC(=O)OCC1c2ccccc2-c2ccccc21...
2,CC(C)(C)OC(=O)N1CCN(C(=O)OCC2c3ccccc3-c3ccccc3...
3,CC(OC(C)(C)C)C(NC(=O)OCC1c2ccccc2-c2ccccc21)C(...
4,COc1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3ccccc32)c1


In [ ]:
ref_mols = pd.concat([mol1,mol2,mol3])

In [ ]:
ref_mols.head()

In [ ]:
ref_mols["buildingblock1_smiles"].unique().shape

In [ ]:
mol1.head()

In [ ]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols = con.query(f"""(SELECT DISTINCT(building_blocks) as building_blocks FROM
((SELECT DISTINCT(buildingblock1_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock2_smiles) as building_blocks
                        FROM parquet_scan('{train_path}'))
                        UNION ALL
                        (SELECT DISTINCT(buildingblock3_smiles) as building_blocks
                        FROM parquet_scan('{train_path}'))))""").df()

con.close()

In [ ]:
mols.head()

In [ ]:
mols.shape

In [ ]:
# Convert SMILES to RDKit molecules
mols['ecfp'] = mols['building_blocks'].apply(Chem.MolFromSmiles)

In [ ]:
mols.rename(columns={"ecfp": "molecule"}, inplace=True)

In [ ]:
# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

In [ ]:
mols.head()

In [ ]:
mols['ecfp'] = mols['molecule'].apply(generate_ecfp)

In [ ]:
mols.head()

In [25]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_brd4 = con.query(f"""(SELECT DISTINCT(building_blocks) as building_blocks FROM
((SELECT DISTINCT(buildingblock1_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'BRD4'
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock2_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'BRD4')
                        UNION ALL
                        (SELECT DISTINCT(buildingblock3_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'BRD4')))""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
mols_brd4.shape

(1145, 1)

In [13]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_brd4_binds = con.query(f"""
                        SELECT COUNT(*)
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'BRD4' AND binds = 1
                        """).df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
mols_brd4_binds

,count_star()
0,456964


In [15]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_brd4_no_binds = con.query(f"""
                        SELECT COUNT(*)
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'BRD4' AND binds = 0
                        """).df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [16]:
mols_brd4_no_binds

,count_star()
0,97958646


In [17]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_hsa_no_binds = con.query(f"""
                        SELECT COUNT(*)
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'HSA' AND binds = 0
                        """).df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
mols_hsa_no_binds

,count_star()
0,98007200


In [19]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_hsa_binds = con.query(f"""
                        SELECT COUNT(*)
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'HSA' AND binds = 1
                        """).df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
mols_hsa_binds

,count_star()
0,408410


In [21]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_seh_binds = con.query(f"""
                        SELECT COUNT(*)
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'sEH' AND binds = 1
                        """).df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [22]:
mols_seh_binds

,count_star()
0,724532


In [23]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_seh_no_binds = con.query(f"""
                        SELECT COUNT(*)
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'sEH' AND binds = 0
                        """).df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [24]:
mols_seh_no_binds

,count_star()
0,97691078


In [26]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_hsa = con.query(f"""(SELECT DISTINCT(building_blocks) as building_blocks FROM
((SELECT DISTINCT(buildingblock1_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'HSA'
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock2_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'HSA')
                        UNION ALL
                        (SELECT DISTINCT(buildingblock3_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'HSA')))""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [28]:
mols_hsa.shape

(1145, 1)

In [29]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_seh = con.query(f"""(SELECT DISTINCT(building_blocks) as building_blocks FROM
((SELECT DISTINCT(buildingblock1_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'sEH'
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock2_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'sEH')
                        UNION ALL
                        (SELECT DISTINCT(buildingblock3_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        WHERE protein_name = 'sEH')))""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [31]:
mols_seh.shape

(1145, 1)

In [37]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_test = con.query(f"""(SELECT DISTINCT(building_blocks) as building_blocks FROM
((SELECT DISTINCT(buildingblock1_smiles) as building_blocks
                        FROM parquet_scan('{test_path}')
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock2_smiles) as building_blocks
                        FROM parquet_scan('{test_path}')
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock3_smiles) as building_blocks
                        FROM parquet_scan('{test_path}')
                        )))""").df()

con.close()

In [38]:
mols_test.shape

(2110, 1)

In [39]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols_train = con.query(f"""(SELECT DISTINCT(building_blocks) as building_blocks FROM
((SELECT DISTINCT(buildingblock1_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock2_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock3_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        )))""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [40]:
mols_train.shape

(1145, 1)

In [41]:
mols_train.head()

,building_blocks
0,Br.Br.NCC1CCCN1c1cccnn1
1,Br.NCc1cccc(Br)n1
2,C=CCOCCCN
3,CC(C)CC(C)(CN)NC(=O)c1cc(Cl)c(Cl)[nH]1.Cl
4,CC(C)NC(=O)NCCN.Cl.Cl


In [42]:
mols_test.head()

,building_blocks
0,O=C(Nc1cc(F)c(F)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21
1,O=C(Nc1cc(F)cc(F)c1C(=O)O)OCC1c2ccccc2-c2ccccc21
2,O=C(Nc1cc(I)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21
3,O=C(Nc1ccc(C(=O)O)c(Cl)c1)OCC1c2ccccc2-c2ccccc21
4,O=C(Nc1ccc(C(=O)O)cc1C(F)(F)F)OCC1c2ccccc2-c2c...


In [44]:
all_mols = pd.concat([mols_train,mols_test])

In [46]:
all_mols.head()

,building_blocks
0,Br.Br.NCC1CCCN1c1cccnn1
1,Br.NCc1cccc(Br)n1
2,C=CCOCCCN
3,CC(C)CC(C)(CN)NC(=O)c1cc(Cl)c(Cl)[nH]1.Cl
4,CC(C)NC(=O)NCCN.Cl.Cl


In [47]:
all_mols["building_blocks"].unique().shape

(2110,)

In [48]:
all_mols["building_blocks"].shape

(3255,)

In [49]:
train_smiles = set(mols_train['building_blocks'])

In [52]:
len(train_smiles)

1145

In [53]:
test_smiles = set(mols_test['building_blocks'])

In [54]:
len(test_smiles)

2110

In [55]:
only_in_train = train_smiles - test_smiles
only_in_train

set()

In [57]:
only_in_test = test_smiles - train_smiles
len(only_in_test)

965

In [58]:
train_smiles

{'Cc1cc(CN)cc(Cl)n1.Cl.Cl',
 'Nc1cc[nH]c(=O)n1',
 'Nc1cc(C(F)(F)F)cc(C(F)(F)F)c1',
 'Cc1cc(O)cc(C)c1N',
 'O=C(Nc1nc(Cl)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21',
 'Cl.Cl.NCCc1nnc2c(=O)[nH]ccn12',
 'Nc1n[nH]c2c(F)c(F)ccc12',
 'Cl.NCC1CCOC2(CCOCC2)C1',
 'CN(C)c1ncccc1CN.Cl.Cl',
 'COc1ccc(CN)cc1C#N.Cl',
 'O=C(N[C@H](Cc1ccc(Cl)cc1)C(=O)O)OCC1c2ccccc2-c2ccccc21',
 'O=C(Nc1ccc(C(=O)O)cc1Cl)OCC1c2ccccc2-c2ccccc21',
 'CC(C)c1nnc([C@H]2C[C@H](CN)[C@H](O)C2)[nH]1',
 'Cc1nccnc1N',
 'Nc1cc(F)cc(F)c1[N+](=O)[O-]',
 'NCCNC(=O)c1cccnc1',
 'Nc1ccc([N+](=O)[O-])cn1',
 'Cl.Cl.NCC#Cc1cccnc1',
 'Cc1cc(CN)c(C)s1',
 'O=C(N[C@@H](Cc1cscn1)C(=O)O)OCC1c2ccccc2-c2ccccc21',
 'COc1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3ccccc32)c1',
 'COc1cnc(N)nc1',
 'N#Cc1nc[nH]c1N',
 'Nc1n[nH]c2cc(F)ccc12',
 'CC1(C)CC(CN)C(C)(C)O1',
 'CCOC(=O)c1c[nH]nc1N',
 'COC(=O)c1cc(Cl)nc(Cl)c1N',
 'NCc1cc(-c2ccccc2)[nH]n1',
 'Cl.Cl.NCc1cnsc1',
 'Nc1ccncn1',
 'Nc1cc(Br)ccn1',
 'Cc1nc(N)ccc1Cl',
 'O=C(N[C@H]1C=C[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2ccccc21',
